Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
    
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 100 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3.905149
Minibatch accuracy: 6.2%
Validation accuracy: 16.7%
Minibatch loss at step 100: 0.892092
Minibatch accuracy: 68.8%
Validation accuracy: 72.0%
Minibatch loss at step 200: 0.878212
Minibatch accuracy: 81.2%
Validation accuracy: 77.6%
Minibatch loss at step 300: 0.401225
Minibatch accuracy: 81.2%
Validation accuracy: 80.0%
Minibatch loss at step 400: 0.240357
Minibatch accuracy: 93.8%
Validation accuracy: 80.7%
Minibatch loss at step 500: 0.670892
Minibatch accuracy: 87.5%
Validation accuracy: 80.7%
Minibatch loss at step 600: 0.293932
Minibatch accuracy: 87.5%
Validation accuracy: 81.5%
Minibatch loss at step 700: 0.914380
Minibatch accuracy: 75.0%
Validation accuracy: 82.1%
Minibatch loss at step 800: 0.723984
Minibatch accuracy: 68.8%
Validation accuracy: 81.7%
Minibatch loss at step 900: 0.644849
Minibatch accuracy: 87.5%
Validation accuracy: 83.4%
Minibatch loss at step 1000: 0.325258
Minibatch accuracy: 93.8%
Validation accuracy: 82.3%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [13]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()
with graph.as_default():
    # Input
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    tf_train_labels  = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)
    
    # Variables
    weights_1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    biases_1  = tf.Variable(tf.zeros([depth]))
    weights_2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    biases_2  = tf.Variable(tf.constant(1.0, shape=[depth]))
    weights_3 = tf.Variable(tf.truncated_normal(
                            [image_size//4 * image_size//4 * depth, num_hidden], stddev=0.1))
    biases_3  = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    weights_4 = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    biases_4  = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Training computation
    def model(data):
        conv   = tf.nn.conv2d(data, weights_1, [1, 1, 1, 1], padding='SAME')
        pool   = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(pool + biases_1)
        conv   = tf.nn.conv2d(hidden, weights_2, [1, 1, 1, 1], padding='SAME')
        pool   = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(pool + biases_2)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, weights_3) + biases_3)
        return tf.matmul(hidden, weights_4) + biases_4
    
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = tf_train_labels, logits=logits))
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
    
    # Predictions
    train_predictions = tf.nn.softmax(logits)
    valid_predictions = tf.nn.softmax(model(tf_valid_dataset))
    test_predictions  = tf.nn.softmax(model(tf_test_dataset))

In [14]:
num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_predictions], feed_dict=feed_dict)
        
        if step % 100 == 0:
            print("Minibatch loss at step {0}: {1:.5f}".format(step, l))
            print("Minibatch accuracy: {0:.1f}%".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {0:.1f}%".format(accuracy(valid_predictions.eval(), valid_labels)))

    print("Test accuracy: {0:.1f}%".format(accuracy(test_predictions.eval(), test_labels)))

Minibatch loss at step 0: 4.59387
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1.35447
Minibatch accuracy: 62.5%
Validation accuracy: 51.4%
Minibatch loss at step 200: 0.80605
Minibatch accuracy: 75.0%
Validation accuracy: 77.5%
Minibatch loss at step 300: 0.35034
Minibatch accuracy: 87.5%
Validation accuracy: 80.5%
Minibatch loss at step 400: 0.32765
Minibatch accuracy: 93.8%
Validation accuracy: 81.6%
Minibatch loss at step 500: 0.71817
Minibatch accuracy: 81.2%
Validation accuracy: 81.4%
Minibatch loss at step 600: 0.36779
Minibatch accuracy: 87.5%
Validation accuracy: 83.1%
Minibatch loss at step 700: 0.77997
Minibatch accuracy: 68.8%
Validation accuracy: 83.7%
Minibatch loss at step 800: 0.56525
Minibatch accuracy: 87.5%
Validation accuracy: 84.2%
Minibatch loss at step 900: 0.66099
Minibatch accuracy: 75.0%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 0.38913
Minibatch accuracy: 87.5%
Validation accuracy: 83.9%
Test accuracy: 90.4%


---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [50]:
batch_size = 256
patch_size = 5
depth_1, depth_2 = 16, 16
hidden_1, hidden_2 = 64, 64

graph = tf.Graph()
with graph.as_default():
    # Input 
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size, image_size, num_channels))
    tf_train_labels  = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset  = tf.constant(test_dataset)
    
    # Variables 
    weights_1 = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth_1], stddev=0.1))
    weights_2 = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth_1, depth_2], stddev=0.1))
    weights_3 = tf.Variable(tf.truncated_normal([image_size//4 * image_size// 4 * depth_2, hidden_1], stddev=0.1))
    weights_4 = tf.Variable(tf.truncated_normal([hidden_1, num_labels], stddev=0.1))
    biases_1  = tf.Variable(tf.zeros([depth_1]))
    biases_2  = tf.Variable(tf.constant(1.0, shape=[depth_2]))
    biases_3  = tf.Variable(tf.constant(1.0, shape=[hidden_1]))
    biases_4  = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    global_step = tf.Variable(0)
    
    # Training computation
    def model(data):
        conv = tf.nn.conv2d(data, weights_1, [1, 1, 1, 1], padding='SAME')
        pool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(pool + biases_1)
        conv = tf.nn.conv2d(hidden, weights_2, [1, 1, 1, 1], padding='SAME')
        pool = tf.nn.max_pool(conv, [1, 2, 2, 1], [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(pool + biases_2)
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1]*shape[2]*shape[3]])
        fc = tf.nn.relu(tf.matmul(reshape, weights_3) + biases_3)
        return tf.matmul(fc, weights_4) + biases_4
    
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)
                         + 1e-3*(tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + 
                                 tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4)))
    
    # optimizer
    learning_rate = tf.train.exponential_decay(0.05, global_step, 4000, 0.65, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    # Predictions
    train_predictions = tf.nn.softmax(logits)
    valid_predictions = tf.nn.softmax(model(tf_valid_dataset))
    test_predictions  = tf.nn.softmax(model(tf_test_dataset))

In [51]:
num_steps = 10001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_predictions], feed_dict=feed_dict)
        
        if step % 1000 == 0:
            print("Minibatch loss at step {0}: {1:.5f}".format(step, l))
            print("Minibatch accuracy: {0:.1f}%".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {0:.1f}%".format(accuracy(valid_predictions.eval(), valid_labels)))

    print("Test accuracy: {0:.1f}%".format(accuracy(test_predictions.eval(), test_labels)))

Minibatch loss at step 0: 3.16528
Minibatch accuracy: 11.7%
Validation accuracy: 9.8%
Minibatch loss at step 1000: 0.58869
Minibatch accuracy: 87.9%
Validation accuracy: 86.3%
Minibatch loss at step 2000: 0.53199
Minibatch accuracy: 88.3%
Validation accuracy: 87.7%
Minibatch loss at step 3000: 0.61527
Minibatch accuracy: 86.7%
Validation accuracy: 88.4%
Minibatch loss at step 4000: 0.53305
Minibatch accuracy: 89.8%
Validation accuracy: 89.1%
Minibatch loss at step 5000: 0.56218
Minibatch accuracy: 88.7%
Validation accuracy: 89.4%
Minibatch loss at step 6000: 0.48131
Minibatch accuracy: 90.2%
Validation accuracy: 89.6%
Minibatch loss at step 7000: 0.37507
Minibatch accuracy: 92.6%
Validation accuracy: 89.9%
Minibatch loss at step 8000: 0.45069
Minibatch accuracy: 89.8%
Validation accuracy: 90.0%
Minibatch loss at step 9000: 0.44150
Minibatch accuracy: 91.4%
Validation accuracy: 90.0%
Minibatch loss at step 10000: 0.41420
Minibatch accuracy: 91.0%
Validation accuracy: 90.2%
Test accuracy

Esse foi o meu melhor resultado: **95.6%**.